In [86]:
import tekore as tk
import pandas as pd
from tqdm.notebook import tqdm

from collections import defaultdict 

CliendId     = '96fb8ea3b71941558b9adc070bca36d6'
ClientSecret = 'b4a837c43e144200ae8d721b59e9203b'

In [2]:
UserToken = tk.prompt_for_user_token(
    CliendId,
    ClientSecret,
    'https://example.com/callback',
    scope=tk.scope.every
)

Spotify = tk.Spotify(UserToken)

UserDisplayName = Spotify.current_user().display_name

Opening browser for Spotify login...
Please paste redirect URL: https://example.com/callback?code=AQB5dQ87q_xS8ylUWxjXx_eTwfs82UqXlcnzONZXh6umjn0TsdsjOgCBvhK2g8DFE-f3N9KsYT8KgZhesCZaviXK1rmnUZ_YjF_CDJlElk2d-UoyGlQ1fdV3gDkBMx4dWj7i8omwU-9TPWxVKh9O4xH5Xdkd7IF7Y6mnRL02qy7CeFZ2yZx1_kKwLWlemdoK8qUEv20LGBOzqe498Pnc8Ts4xCBbzNEIqfbFiImsxh1gxnxHwM5u-NDgiDqdkwop5O6i7oHuNcv3jT5w2cBKrH4GX0uKcd9qwLuCGQ-33OrtzotEM7BEEjCjPkV_a1iPZLrlUSOPuSEB17ohpvI8QjUdIO2dx33iGz4b1YDHSGleOtT3OsL-5CMCi89iOIr3MjxWXlgcY3tfigOWlFE2M45ZHWqKUYR82hY3wgXX26_qahTVXDvMjCZHSMUND94vrNSEdW2z3fhfYWEKwZnb1vuPqPqg23EB1iRAY08zUxE42A4AbsFVzdB7CbuI2A0mcjxWuOCvappQ45u3Ccbl5ywYSk0xydxCxNDNL2W4z1UzPJXvpO5uNz2hrAX6nmY-KBC5pYMd39oGzbuAvyGqHlmWY26L9RyWgNSVSOypou3EdVIpKGVTBVt_YilLCPVUarmRgOQdK9hRZbuu3EUNGL8DxRnJhOhGk_o5K4pAocjgsoU&state=aFoUBwXVidhI6HbhgYmjJ1Ud2EXqlUgL4F0QEqLgafs


# Helper Functions

In [41]:
def GetTracksForGenre(LookupGenre):
    TracksForGenre = []
    
    for Track in Tracks:
        for Artist in Track.track.artists:
            Genre = ArtistGenres[Artist.id]
            
            if Genre != 'no genre':
                break
        
        if Genre == LookupGenre:
            TrackArtists = ', '.join([x.name for x in Track.track.artists])
            TracksForGenre += ['{} - {}'.format(Track.track.name, TrackArtists)]
    
    return TracksForGenre

# Create Genre Playlists

In [43]:
Tracks = [x for x in Spotify.all_items(Spotify.saved_tracks())]
ArtistGenres = {}

In [44]:
Data = defaultdict(list)
Artists = set()

# Get all of the unique artists in the tracks
for Track in Tracks:
    for Artist in Track.track.artists:
        Artists.add(Artist.id)

Artists = list(Artists)

# Batch the artists before looking them up
BatchSize = 50
ArtistIdBatches = [Artists[i:i+BatchSize] for i in range(0, len(Artists), BatchSize)]

# Look up and record each artist's genre
for Batch in ArtistIdBatches:
    Artists = Spotify.artists(Batch)
    
    for Artist in Artists:
        Genre = Artist.genres[0] if len(Artist.genres) > 0 else 'no genre'
        ArtistGenres[Artist.id] = Genre

In [59]:
# Come up with one track per genre
GenreTracks = {}
GenreCount = defaultdict(int)

for Track in Tracks:
    for Artist in Track.track.artists:
        Genre = ArtistGenres[Artist.id]
        
        if Genre != 'no genre':
            break
    
    GenreCount[Genre] += 1
    if Genre not in GenreTracks:
        GenreTracks[Genre] = Track.track.name    
        
GenreTracks = {'Genre' : list(GenreTracks.keys()), 'Count' : list(GenreCount.values()), 'Track' : list(GenreTracks.values())}

In [60]:
# Make sure you have a mapping for all genres
AllGenres = set(ArtistGenres.values())

AllGenreMapping = pd.DataFrame(GenreTracks)
AllGenreMapping = AllGenreMapping.set_index('Genre')

In [68]:
# Export new genres to csv to be updated
ExistingGenreMapping = pd.read_csv('GenreMapping.csv', index_col='Genre')

AllGenreMappingWithKeys = AllGenreMapping.join(ExistingGenreMapping['Mapping'])
AllGenreMappingWithKeys.to_csv('GenreMapping.csv')

print('There are {} genres that need to be mapped'.format(AllGenreMappingWithKeys['Mapping'].isna().sum()))

There are 0 genres that need to be mapped


In [80]:
FullGenreMapping = pd.read_csv('GenreMapping.csv', index_col='Genre')

assert(AllGenreMapping.isna().sum().sum() == 0), 'Some genres are not mapped.'

In [76]:
# Create playlists for genre mapping
ExistingPlaylists = Spotify.all_items(Spotify.followed_playlists())

MappingPlaylistId = dict.fromkeys(FullGenreMapping['Mapping'], None)

# Get destination playlist for each mapping
for Playlist in ExistingPlaylists:
    if Playlist.owner.display_name == UserDisplayName:

        if Playlist.name in MappingPlaylistId:
            assert(MappingPlaylistId[Playlist.name] is None), f'Playlist {playlist.name} has multiple mappings.'
            
            MappingPlaylistId[Playlist.name] = Playlist.id

# Make sure no genre mapping is missing a playlist
for k, v in MappingPlaylistId.items():
    if v is None:
        Playlist = Spotify.playlist_create(Spotify.current_user().id,
                                            k,
                                            False,
                                            '''
                                            A playlist created by SpotifyPlaylistBuilder to house all 
                                            of your tracks that are in the {} genre
                                            '''.format(k))
        
        MappingPlaylistId[k] = Playlist.id

In [77]:
# Clear all genre mapping playlists
for PlaylistId in MappingPlaylistId.values():
    Spotify.playlist_clear(PlaylistId)

In [81]:
# Get a list of tracks to add for each genre
CategoryTracks = {c : [] for c in FullGenreMapping['Mapping'].drop_duplicates().values}

TrackData = {k : [] for k in ['Track', 'Genre', 'Category']}

for Track in Tracks:
    Genre = ArtistGenres[Track.track.artists[0].id]
    Category = FullGenreMapping.loc[Genre]['Mapping']
    CategoryTracks[Category] += [Track.track.uri]
    
    TrackData['Track'] += [Track.track.name]
    TrackData['Genre'] += [Genre]
    TrackData['Category'] += [Category]
    
TrackData = pd.DataFrame(TrackData)
TrackData.to_csv('TrackData.csv')

In [88]:
for c, ts in tqdm(CategoryTracks.items()):
    PlaylistId = MappingPlaylistId[c]

    # Batch the artists before looking them up
    BatchSize = 50
    TrackUriBatches = [ts[i:i+BatchSize] for i in range(0, len(ts), BatchSize)]
    
    for Batch in tqdm(TrackUriBatches, leave=False):
        Spotify.playlist_add(PlaylistId, Batch)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]